<a href="https://colab.research.google.com/github/vidyabandgar97/A2A/blob/main/detectron2_500_07_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/detectron2_500/

/content/drive/MyDrive/detectron2_500


In [3]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8 MB 23 kB/s 
     |████████████████████████████████| 6.6 MB 42.1 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.5.0+cu101 which is incompatible.
     |████████████████████████████████| 274 kB 12.9 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=eecb4160e8e1119c6ecb7890c51f6

In [4]:
!pip install torch==1.9.0 torchvision==0.4.1

     |████████████████████████████████| 831.4 MB 2.7 kB/s 
     |████████████████████████████████| 10.1 MB 19.5 MB/s 
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install torch==1.9.0 and torchvision==0.4.1 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch==1.9.0
    torchvision 0.4.1 depends on torch==1.3.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/user_guide/#fixing-conflicting-dependencies


In [4]:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
     |████████████████████████████████| 6.2 MB 3.0 MB/s 
     |████████████████████████████████| 49 kB 5.0 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20210825-py3-none-any.whl size=60661 sha256=d40b7f6c3f39aad6749b91e27855a38a569fb024430d55fa2ceeecb7b1b6de1f
  Stored in directory: /root/.cache/pip/wheels/53/c4/f8/c4cb07f135845218b019b4a55d8a0470a0f21ee13f8dcd16be
Successfully built fvcore


In [5]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer

In [6]:
#if your dataset is in COCO format, this cell can be replaced by the following three lines:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train2", {}, "/content/drive/MyDrive/detectron2_500/train.json", "/content/drive/MyDrive/detectron2_500/Dataset")
register_coco_instances("my_dataset_val2", {}, "/content/drive/MyDrive/detectron2_500/test.json", "/content/drive/MyDrive/detectron2_500/test_data")
register_coco_instances("my_dataset_test2", {}, "/content/drive/MyDrive/detectron2_500/test.json", "/content/drive/MyDrive/detectron2_500/test_data")

In [7]:
#We are importing our own Trainer Module here to use the COCO validation evaluation during training. Otherwise no validation eval occurs.

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.data import transforms as T

from detectron2.data import (
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)

from detectron2.data import DatasetMapper   # the default mapper

In [8]:
class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_train_loader(cls, cfg):
    return build_detection_train_loader(
        cfg, 
        mapper=DatasetMapper(
            cfg, 
            is_train=True, 
            augmentations=[
                           T.Resize((1024, 1024)), 
                           T.HFlipTransform(1024)
                           ]
        )
    )

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [10]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
import os

cfg = get_cfg()
cfg.OUTPUT_DIR = "/content/drive/MyDrive/detectron2_500/output_detect"
cfg.SOLVER.CHECKPOINT_PERIOD = 1000
cfg.merge_from_file(model_zoo.get_config_file("Misc/mask_rcnn_R_50_FPN_3x_dconv_c3-c5.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train2",)
cfg.DATASETS.TEST = ("my_dataset_val2",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("Misc/mask_rcnn_R_50_FPN_3x_dconv_c3-c5.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 86000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()
#trainer.test()


[09/07 09:56:42 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [11]:
import os
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/detectron2_500/output_detect/model_final.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_val2", cfg, False, output_dir="/content/drive/MyDrive/detectron2_500/output_detect/")
val_loader = build_detection_test_loader(cfg, "my_dataset_val2")
inference_on_dataset(trainer.model, val_loader, evaluator)

[09/07 09:57:04 d2.data.datasets.coco]: Loaded 190 images in COCO format from /content/drive/MyDrive/detectron2_500/test.json
[09/07 09:57:04 d2.data.build]: Distribution of instances among all 6 categories:
|  category  | #instances   |  category  | #instances   |   category    | #instances   |
|:----------:|:-------------|:----------:|:-------------|:-------------:|:-------------|
| aeroplane  | 100          |    car     | 99           |     chair     | 99           |
|    cow     | 100          |   person   | 100          | traffic_light | 99           |
|            |              |            |              |               |              |
|   total    | 597          |            |              |               |              |
[09/07 09:57:04 d2.data.common]: Serializing 190 elements to byte tensors and concatenating them all ...
[09/07 09:57:04 d2.data.common]: Serialized dataset takes 0.34 MiB
[09/07 09:57:04 d2.evaluation.evaluator]: Start inference on 190 images
[09/07 09:57:0

OrderedDict([('bbox',
              {'AP': 28.98399615813917,
               'AP-aeroplane': 54.57337962218733,
               'AP-car': 14.66612805636027,
               'AP-chair': 19.969412071910146,
               'AP-cow': 49.021736761204615,
               'AP-person': 10.650646989033767,
               'AP-traffic_light': 25.022673448138853,
               'AP50': 45.03740027896978,
               'AP75': 29.703910273555806,
               'APl': 43.91260835680967,
               'APm': 40.464182486473106,
               'APs': 14.5215639775658}),
             ('segm',
              {'AP': 23.25659772128245,
               'AP-aeroplane': 39.12518787892412,
               'AP-car': 13.695654512165826,
               'AP-chair': 14.429427753456547,
               'AP-cow': 40.66808307048525,
               'AP-person': 8.839691185693024,
               'AP-traffic_light': 22.781541926969926,
               'AP50': 42.384749137878536,
               'AP75': 25.316161950387716,
   

In [12]:
import os
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/detectron2_500/output_detect/model_0085999.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_val2", cfg, False, output_dir="/content/drive/MyDrive/detectron2_500/output_detect/")
val_loader = build_detection_test_loader(cfg, "my_dataset_val2")
inference_on_dataset(trainer.model, val_loader, evaluator)

[09/07 09:59:55 d2.data.datasets.coco]: Loaded 190 images in COCO format from /content/drive/MyDrive/detectron2_500/test.json
[09/07 09:59:55 d2.data.common]: Serializing 190 elements to byte tensors and concatenating them all ...
[09/07 09:59:55 d2.data.common]: Serialized dataset takes 0.34 MiB
[09/07 09:59:55 d2.evaluation.evaluator]: Start inference on 190 images
[09/07 09:59:59 d2.evaluation.evaluator]: Inference done 11/190. 0.3358 s / img. ETA=0:01:01
[09/07 10:00:05 d2.evaluation.evaluator]: Inference done 26/190. 0.3456 s / img. ETA=0:00:57
[09/07 10:00:10 d2.evaluation.evaluator]: Inference done 40/190. 0.3519 s / img. ETA=0:00:53
[09/07 10:00:15 d2.evaluation.evaluator]: Inference done 55/190. 0.3510 s / img. ETA=0:00:48
[09/07 10:00:20 d2.evaluation.evaluator]: Inference done 69/190. 0.3519 s / img. ETA=0:00:43
[09/07 10:00:25 d2.evaluation.evaluator]: Inference done 83/190. 0.3526 s / img. ETA=0:00:38
[09/07 10:00:31 d2.evaluation.evaluator]: Inference done 97/190. 0.3543 

OrderedDict([('bbox',
              {'AP': 28.98399615813917,
               'AP-aeroplane': 54.57337962218733,
               'AP-car': 14.66612805636027,
               'AP-chair': 19.969412071910146,
               'AP-cow': 49.021736761204615,
               'AP-person': 10.650646989033767,
               'AP-traffic_light': 25.022673448138853,
               'AP50': 45.03740027896978,
               'AP75': 29.703910273555806,
               'APl': 43.91260835680967,
               'APm': 40.464182486473106,
               'APs': 14.5215639775658}),
             ('segm',
              {'AP': 23.25659772128245,
               'AP-aeroplane': 39.12518787892412,
               'AP-car': 13.695654512165826,
               'AP-chair': 14.429427753456547,
               'AP-cow': 40.66808307048525,
               'AP-person': 8.839691185693024,
               'AP-traffic_light': 22.781541926969926,
               'AP50': 42.384749137878536,
               'AP75': 25.316161950387716,
   

In [13]:
import os
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/detectron2_500/output_detect/model_final.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_train2", cfg, False, output_dir="/content/drive/MyDrive/detectron2_500/output_detect/")
val_loader = build_detection_test_loader(cfg, "my_dataset_train2")
inference_on_dataset(trainer.model, val_loader, evaluator)

[09/07 10:01:50 d2.data.datasets.coco]: Loaded 689 images in COCO format from /content/drive/MyDrive/detectron2_500/train.json
[09/07 10:01:51 d2.data.common]: Serializing 689 elements to byte tensors and concatenating them all ...
[09/07 10:01:51 d2.data.common]: Serialized dataset takes 1.35 MiB
[09/07 10:01:51 d2.evaluation.evaluator]: Start inference on 689 images
[09/07 10:01:56 d2.evaluation.evaluator]: Inference done 11/689. 0.3721 s / img. ETA=0:04:15
[09/07 10:02:01 d2.evaluation.evaluator]: Inference done 26/689. 0.3502 s / img. ETA=0:03:55
[09/07 10:02:06 d2.evaluation.evaluator]: Inference done 41/689. 0.3476 s / img. ETA=0:03:49
[09/07 10:02:11 d2.evaluation.evaluator]: Inference done 55/689. 0.3501 s / img. ETA=0:03:46
[09/07 10:02:17 d2.evaluation.evaluator]: Inference done 69/689. 0.3517 s / img. ETA=0:03:43
[09/07 10:02:22 d2.evaluation.evaluator]: Inference done 83/689. 0.3539 s / img. ETA=0:03:39
[09/07 10:02:27 d2.evaluation.evaluator]: Inference done 97/689. 0.3543

OrderedDict([('bbox',
              {'AP': 90.14414637284705,
               'AP-aeroplane': 97.5046735442775,
               'AP-car': 87.76866864133054,
               'AP-chair': 89.47432734229967,
               'AP-cow': 90.4748976867685,
               'AP-person': 90.23886735543314,
               'AP-traffic_light': 85.40344366697295,
               'AP50': 98.73068806907918,
               'AP75': 97.52827898558189,
               'APl': 96.084273558184,
               'APm': 93.53585458883222,
               'APs': 87.73384424280555}),
             ('segm',
              {'AP': 71.0479835296563,
               'AP-aeroplane': 72.37761169743177,
               'AP-car': 73.34421383141503,
               'AP-chair': 68.96028807350005,
               'AP-cow': 66.68790716391861,
               'AP-person': 69.0876560609309,
               'AP-traffic_light': 75.83022435074153,
               'AP50': 97.70041956815953,
               'AP75': 85.07104841918662,
               'APl

In [14]:
import os
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/detectron2_500/output_detect/model_0085999.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_train2", cfg, False, output_dir="/content/drive/MyDrive/detectron2_500/output_detect/")
val_loader = build_detection_test_loader(cfg, "my_dataset_train2")
inference_on_dataset(trainer.model, val_loader, evaluator)

[09/07 10:11:34 d2.data.datasets.coco]: Loaded 689 images in COCO format from /content/drive/MyDrive/detectron2_500/train.json
[09/07 10:11:34 d2.data.common]: Serializing 689 elements to byte tensors and concatenating them all ...
[09/07 10:11:34 d2.data.common]: Serialized dataset takes 1.35 MiB
[09/07 10:11:34 d2.evaluation.evaluator]: Start inference on 689 images
[09/07 10:11:38 d2.evaluation.evaluator]: Inference done 11/689. 0.3663 s / img. ETA=0:04:11
[09/07 10:11:43 d2.evaluation.evaluator]: Inference done 26/689. 0.3474 s / img. ETA=0:03:53
[09/07 10:11:49 d2.evaluation.evaluator]: Inference done 41/689. 0.3452 s / img. ETA=0:03:47
[09/07 10:11:54 d2.evaluation.evaluator]: Inference done 55/689. 0.3478 s / img. ETA=0:03:43
[09/07 10:11:59 d2.evaluation.evaluator]: Inference done 69/689. 0.3492 s / img. ETA=0:03:40
[09/07 10:12:04 d2.evaluation.evaluator]: Inference done 83/689. 0.3514 s / img. ETA=0:03:37
[09/07 10:12:09 d2.evaluation.evaluator]: Inference done 97/689. 0.3519

OrderedDict([('bbox',
              {'AP': 90.14414637284705,
               'AP-aeroplane': 97.5046735442775,
               'AP-car': 87.76866864133054,
               'AP-chair': 89.47432734229967,
               'AP-cow': 90.4748976867685,
               'AP-person': 90.23886735543314,
               'AP-traffic_light': 85.40344366697295,
               'AP50': 98.73068806907918,
               'AP75': 97.52827898558189,
               'APl': 96.084273558184,
               'APm': 93.53585458883222,
               'APs': 87.73384424280555}),
             ('segm',
              {'AP': 71.0479835296563,
               'AP-aeroplane': 72.37761169743177,
               'AP-car': 73.34421383141503,
               'AP-chair': 68.96028807350005,
               'AP-cow': 66.68790716391861,
               'AP-person': 69.0876560609309,
               'AP-traffic_light': 75.83022435074153,
               'AP50': 97.70041956815953,
               'AP75': 85.07104841918662,
               'APl